**ClusterRep_v3_behvPaper_dataRearrange.ipynb**


In [2]:
import numpy as np
import scipy as sp
import seaborn as sns
import pandas as pd
import scipy
import sklearn
import os
import mne
from matplotlib import pyplot as plt
from scipy.io import loadmat


In [2]:
np.warnings.filterwarnings('ignore')

**Define the functions**

In [2]:
def angle_to_onezero(tgtXY, zeroXY, centerXY):
    """
    calculate the angle between a target data point and zero point (centerX+RChunkPX, centerY)
    """
    tgtX, tgtY       = tgtXY
    zeroX, zeroY     = zeroXY
    centerX, centerY = centerXY
    Col1    = np.array([tgtX, tgtY]) - np.array([centerX, centerY]) # Col1, Col2 = (n,)
    Col2    = np.array([zeroX, zeroY]) - np.array([centerX, centerY])
    rad_t2o = np.arccos(np.dot(Col1, Col2) / (np.linalg.norm(Col1) * np.linalg.norm(Col2)))
    if tgtY <= centerY:
        rad_t2o = -rad_t2o
    else: 
        rad_t2o = rad_t2o
    return rad_t2o

In [3]:
def node_which_cluster(nodeId, clsDef):
    """
    which cluster a specific node belongs to
    """
    cluster_in = np.zeros((np.shape(clsDef)[0], 1))
    for i in range(0, np.shape(clsDef)[0]):
        cluster_in[i] = np.sum(nodeId == clsDef[i, :])
    return cluster_in

In [4]:
def ismember(A, B):
    return [ np.sum(a == B) for a in A ]

In [3]:
tgtNum  = 1
dtrNums = np.array([1, 2, 3])
nodeNum = 15
clsDef  = np.array([[1, 2, 3, 4, 5], 
                    [6, 7, 8, 9, 10], 
                    [11, 12, 13, 14, 15]])

def LynnNetwork():
    transMat = np.zeros((nodeNum, nodeNum))
    transMat[np.array([2, 3, 4, 15])-1, 1-1] = 0.25
    transMat[np.array([1, 3, 4, 5])-1, 2-1]  = 0.25
    transMat[np.array([1, 2, 4, 5])-1, 3-1]  = 0.25
    transMat[np.array([1, 2, 3, 5])-1, 4-1]  = 0.25
    transMat[np.array([2, 3, 4, 6])-1, 5-1]  = 0.25
    
    transMat[np.array([5, 7, 8, 9])-1, 6-1]   = 0.25
    transMat[np.array([6, 8, 9, 10])-1, 7-1]  = 0.25
    transMat[np.array([6, 7, 9, 10])-1, 8-1]  = 0.25
    transMat[np.array([6, 7, 8, 10])-1, 9-1]  = 0.25
    transMat[np.array([7, 8, 9, 11])-1, 10-1] = 0.25
    
    transMat[np.array([10, 12, 13, 14])-1, 11-1] = 0.25
    transMat[np.array([11, 13, 14, 15])-1, 12-1] = 0.25
    transMat[np.array([11, 12, 14, 15])-1, 13-1] = 0.25
    transMat[np.array([11, 12, 13, 15])-1, 14-1] = 0.25
    transMat[np.array([1, 12, 13, 14])-1, 15-1]  = 0.25 
    return transMat
transMat = LynnNetwork()

In [ ]:
# detecting communities from the community network

In [4]:
import networkx as nx
from networkx.algorithms import community

In [5]:
G = nx.Graph()
G.add_nodes_from([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
# community 1
G.add_edges_from([(1, 2), (1, 3), (1, 4), (1, 15)])
G.add_edges_from([(2, 1), (2, 3), (2, 4), (2, 5)])
G.add_edges_from([(3, 1), (3, 2), (3, 4), (3, 5)])
G.add_edges_from([(4, 1), (4, 2), (4, 3), (4, 5)])
G.add_edges_from([(5, 2), (5, 3), (5, 4), (5, 6)])
# community 2
G.add_edges_from([(6, 7), (6, 8), (6, 9), (6, 5)])
G.add_edges_from([(7, 6), (7, 8), (7, 9), (7, 10)])
G.add_edges_from([(8, 6), (8, 7), (8, 9), (8, 10)])
G.add_edges_from([(9, 6), (9, 7), (9, 8), (9, 10)])
G.add_edges_from([(10, 7), (10, 8), (10, 9), (10, 11)])
# community 3
G.add_edges_from([(11, 12), (11, 13), (11, 14), (11, 10)])
G.add_edges_from([(12, 11), (12, 13), (12, 14), (12, 15)])
G.add_edges_from([(13, 11), (13, 12), (13, 14), (13, 15)])
G.add_edges_from([(14, 11), (14, 12), (14, 13), (14, 15)])
G.add_edges_from([(15, 12), (15, 13), (15, 14), (15, 1)])


In [6]:
# mininal distance between any two nodes
length = nx.all_pairs_shortest_path_length(G)
length = list(length)

In [7]:
# convert the length into a matrix
nStates = nodeNum
minD_mat = np.zeros((nStates, nStates), dtype=int)
for i in range(0, nStates):
    fromNode_idx = np.array(length[i])[0]   # the 1st column: the index of the from node
    minD_list    = np.array(length[i])[1]   # the 2nd column: a {}
    toNode_idx   = list(minD_list)          # index of the to node
    toNode_minD  = list(minD_list.values()) # the minimal distance
    for j in range(0, nStates):
        minD_mat[toNode_idx[j]-1, fromNode_idx-1] = toNode_minD[j]
minD_mat

array([[0, 1, 1, 1, 2, 3, 4, 4, 4, 4, 3, 2, 2, 2, 1],
       [1, 0, 1, 1, 1, 2, 3, 3, 3, 4, 4, 3, 3, 3, 2],
       [1, 1, 0, 1, 1, 2, 3, 3, 3, 4, 4, 3, 3, 3, 2],
       [1, 1, 1, 0, 1, 2, 3, 3, 3, 4, 4, 3, 3, 3, 2],
       [2, 1, 1, 1, 0, 1, 2, 2, 2, 3, 4, 4, 4, 4, 3],
       [3, 2, 2, 2, 1, 0, 1, 1, 1, 2, 3, 4, 4, 4, 4],
       [4, 3, 3, 3, 2, 1, 0, 1, 1, 1, 2, 3, 3, 3, 4],
       [4, 3, 3, 3, 2, 1, 1, 0, 1, 1, 2, 3, 3, 3, 4],
       [4, 3, 3, 3, 2, 1, 1, 1, 0, 1, 2, 3, 3, 3, 4],
       [4, 4, 4, 4, 3, 2, 1, 1, 1, 0, 1, 2, 2, 2, 3],
       [3, 4, 4, 4, 4, 3, 2, 2, 2, 1, 0, 1, 1, 1, 2],
       [2, 3, 3, 3, 4, 4, 3, 3, 3, 2, 1, 0, 1, 1, 1],
       [2, 3, 3, 3, 4, 4, 3, 3, 3, 2, 1, 1, 0, 1, 1],
       [2, 3, 3, 3, 4, 4, 3, 3, 3, 2, 1, 1, 1, 0, 1],
       [1, 2, 2, 2, 3, 4, 4, 4, 4, 3, 2, 1, 1, 1, 0]])

In [8]:
# calculate communities
communities = community.greedy_modularity_communities(G)
communities

[frozenset({1, 2, 3, 4, 5}),
 frozenset({6, 7, 8, 9, 10}),
 frozenset({11, 12, 13, 14, 15})]

# Experiment 1-3: (1) Exp1: implicit learning + mouse trajectory + sequential display; (2) Exp 2: explicit learning + mouse trajectory + sequential display; (3) Exp 3: implicit learning + mouse trajectory + shuffled display for Random Walk. Distractor numbers in all 3 experiments: 1/2/3 

In [6]:
# Experiment settings: iMac
sampleRate     = 60
scrWidthCM     = 47.9
scrHeightCM    = 26.9
scrWidthPX     = 2048
scrHeightPX    = 1152
centerX        = scrWidthPX / 2
centerY        = scrHeightPX / 2
visualAngle    = 9
visualAngleCir = (visualAngle / 180) * np.pi
visualDist     = 86
dispDist       = 2 * visualDist * np.tan(visualAngleCir / 2)
RChunk         = dispDist / 2 # radias of the invisible circle
cm2pxV         = scrHeightPX / scrHeightCM
cm2pxH         = scrWidthPX / scrWidthCM
RChunkPX       = RChunk * cm2pxV
zeroPos        = np.array([centerX + RChunkPX, centerY + 0])
# (2) time settings
fixationDur = 0.6
respDur     = 2.0
grayoffT    = 0.8
grayOnT     = 0.8
bonusDec    = 0.5
bonusTime   = 1 
# (3) trial settings
rndTrial    = 700
HamTrial    = 800
nTrials     = rndTrial + HamTrial
rndInter    = 85
HamInter    = 15
# experiment 1.1-1.3
#rndArray    = np.array([230, 2*230, 2*230+240], dtype=int) # ..................The 1st 3 blocks were random pathways
# experiment 1.4
rndArray    = np.array([200, 2*200, 3*200, 3*200+100], dtype=int)
HamArray    = np.asarray(np.array(np.linspace(700+200, 1500, 4, dtype=int))) # The last 4 blocks were random + hamiltonian mixtures; !! np.asarray(): convert a sequence into array !!
tInBlc      = np.concatenate((rndArray, HamArray))
nBlock      = len(tInBlc)
trialsInBlc = np.zeros((nBlock, 2), dtype=int)
trialsInBlc[0, 0]  = 1 - 1
trialsInBlc[1:, 0] = tInBlc[:-1]
trialsInBlc[:, 1]  = tInBlc

# Behavioral data column index
rndOrHam = 0     # trial countings for the random (1-700) and hamiltonian (1-800) walk
blockNo  = 1     # block number
trialNo  = 2     # trial number
trialTp  = 3     # trial type: 1-random; 2-hamiltonian
objTgt   = 4     # target object
objResp  = np.array([5, 6, 7, 8]) # responded objects
moveTime = 9     # moving time of the mouse
respRT   = 10    # response reaction time
dispRT   = 11    # object total display time
errorIdx = 12    # whether error (including incorrect response and time out) or not: 1-yes
timeOIdx = 13    # whether time out: 1-yes
bonusCn  = 14    # bonus in each trial
trialLt  = 15    # trial total length
imgPosX  = np.array([16, 17, 18, 19])
imgPosY  = np.array([20, 21, 22, 23])
fastIdx  = 24

folder   = '/Users/ren/Projects-NeuroCode/MyExperiment/HierarchicalCluster'

**Experiment 1: implicit learning + mouse trajectory + sequential display**

In [7]:
# Read the mouse trajectory for each block
data_behv   = folder + '/FormalExp-LynnNetwork-Results/' 
subj_list   = ['wsn_1_f_18', 'dy_2_f_22', 'haq_3_f_24', 'hry_4_f_20', 'zjx_5_m_20', 'yyq_6_f_18', 'zkw_7_m_18', 'zy_8_f_20', 'hys_9_m_20', 'cjj_10_m_18', 
               'dwq_11_f_22', 'ljl_12_m_20', 'jyx_13_m_19', 'zk_14_f_21', 'lsy_15_m_19', 'cjl_16_m_19', 'yjy_17_f_23', 'lym_18_f_19', 'pr_19_f_23', 'ws_20_f_21',
               'wn_21_f_21', 'hjy_22_f_18', 'qyk_23_f_22', 'yd_24_f_20']
subjLab     = ['wsn1', 'dy2', 'haq3', 'hry4', 'zjx5', 'yyq6', 'zkw7', 'zy8', 'hys9', 'cjj10', 
               'dwq11', 'ljl12', 'jyx13', 'zk14', 'lsy15', 'cjl16', 'yjy17', 'lym18', 'pr19', 'ws20', 
               'wn21', 'hjy22', 'qyk23', 'yd24']
subj_listBv = subj_list
subLen      = len(subj_list)

**Experiment 2: explicit learning + mouse trajectory + sequential display**

In [8]:
data_behv   = folder + '/FormalExp-ExplicitLearning-LynnNetwork-Results/' 
subj_list   = ['hsp_1_m_21', 'hr_2_f_22', 'pxy_3_m_19', 'wyx_4_m_22', 'lml_5_f_19', 'lf_6_m_21', 'md_7_f_21', 'srz_8_m_20', 'fsq_9_f_18', 'caq_10_f_20',
               'xjm_11_f_26', 'cjs_12_m_19', 'xxx_13_f_19', 'wsq_14_f_19', 'zzm_15_f_18', 'lsy_16_f_20', 'man_17_f_19', 'zxy_18_f_18', 'cyh_19_f_21', 'szn_20_f_19',
               'sb_21_m_22', 'drq_22_f_18', 'cy_23_f_18', 'zhc_24_m_21']
subjLab     = ['hsp1', 'hr2', 'pxy3', 'wyx4', 'lml5', 'lf6', 'md7', 'srz8', 'fsq9', 'caq10', 
               'xjm11', 'cjs12', 'xxx13', 'wsq14', 'zzm15', 'lsy16', 'man17', 'zxy18', 'cyh19', 'szn20',
               'sb21', 'drq22', 'cy23', 'zhc24']
subj_listBv = subj_list
subLen      = len(subj_list)

**Experiment 3: implicit learning + mouse trajectory + shuffled display for Random Walk**

In [8]:
data_behv   = folder + '/FormalExp-LynnNetwork-ImplicitRandom-Results/' 
subj_list   = ['zyh_1_f_21', 'why_2_m_18', 'cr_3_f_19', 'zyx_4_f_25', 'wym_5_m_23', 'wd_6_f_18', 'lyh_7_f_21', 'zr_8_f_24', 'zyh_9_m_19', 'zzy_10_f_19', 
               'smq_11_f_25', 'sz_12_f_20', 'lzy_13_m_22', 'yxy_14_f_18', 'zxl_15_f_18', 'wqh_16_m_23', 'zxj_17_m_21', 'skx_18_f_20', 'zlh_19_m_24', 'gwt_20_f_23', 
               'lwn_21_f_18', 'lrp_22_m_21', 'sjj_23_f_18', 'xy_24_f_19']
subjLab     = ['zyh1', 'why2', 'cr3', 'zyx4', 'wym5', 'wd6', 'lyh7', 'zr8', 'lyh9', 'zzy10',
               'smq11', 'sz12', 'lzy13', 'yxy14', 'zxl15', 'wqh16', 'zxj17', 'skx18', 'zlh19', 'gwt20',
               'lwn21', 'lrp22', 'sjj23', 'xy24']
subj_listBv = subj_list
subLen      = len(subj_list)

## Reorganize clusterResult.mat

In [9]:
# convert the clusterResult.mat into dataframes
for SubIdx in range(0, subLen):
    subjBv  = subj_listBv[SubIdx]
    subTit  = subjLab[SubIdx]
    subjDir = data_behv + subjBv + '/'
    # load the 'respYes_trials_blc' and 'mouseTraj_trials_blc' in order to label the trials according to their correctness and response times
    mouseTraj_cell   = loadmat(subjDir + subjBv + 'mouseTraj_trials_blc.mat')
    mouseTraj_trials = mouseTraj_cell['mouseTraj_trials_blc'] # np.shape(mouseTraj_trials)
    respYes_cell     = loadmat(subjDir + subjBv + 'respYes_trials_blc.mat')
    respYes_trials   = respYes_cell['respYes_trials_blc']
    # load the 'clusterResult.mat' which stores the x- and y-position of target image 
    # behvMat = loadmat(subjDir + subjBv + 'clusterResult_Blocks.mat') # 'clusterResult'
    behvMat = loadmat(subjDir + subjBv + 'clusterResult_reOrg.mat') # 'clusterResult'
    clusterResult = behvMat['clusterResult']
    clusterResult = pd.DataFrame(clusterResult) 
    trials_Col    = range(clusterResult.shape[0])
    dtrNo_Col     = clusterResult.iloc[trials_Col, objResp]
    # Labeling whether the from- and to-nodes are boundary nodes
    from_nodes    = clusterResult.iloc[trials_Col, objTgt]
    to_nodes      = clusterResult.iloc[trials_Col, objResp[0]]
    fromNd_Col    = np.zeros((clusterResult.shape[0], 1))
    toNd_Col      = np.zeros((clusterResult.shape[0], 1))
    fromNd_Col[np.logical_or.reduce((from_nodes==1, from_nodes==5, from_nodes==6, from_nodes==10, from_nodes==11, from_nodes==15))] = 1
    toNd_Col[np.logical_or.reduce((to_nodes==1, to_nodes==5, to_nodes==6, to_nodes==10, to_nodes==11, to_nodes==15))]               = 1
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = fromNd_Col)
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = toNd_Col)
    # Labeling whether there is a distractor that belongs to the same cluster as the target
    dtrNoAll      = np.zeros((clusterResult.shape[0], 1))
    dtrInCls      = np.zeros((clusterResult.shape[0], 1))
    transTpAll    = np.zeros((clusterResult.shape[0], 1))
    labCorAll     = np.zeros((clusterResult.shape[0], 1))
    labCorTimeAll = np.zeros((clusterResult.shape[0], 1))
    recencyNodes  = np.zeros((clusterResult.shape[0], 2))
    for ii in range(0, clusterResult.shape[0]):
        # Number of distractor numbers
        dtrNo_ii     = clusterResult.iloc[ii, objResp]
        dtrNo_cnt    = np.shape(dtrNo_ii[dtrNo_ii != 0])[0] - 1
        dtrNoAll[ii] = dtrNo_cnt
        # transition types: (1) within-within; (2) within-between; (3) between-within; (4) between-between
        if fromNd_Col[ii] == 0 and toNd_Col[ii] == 0:
            transType = 0
        elif fromNd_Col[ii] == 0 and toNd_Col[ii] == 1:
            transType = 1
        elif fromNd_Col[ii] == 1 and toNd_Col[ii] == 0:
            transType = 2
        elif fromNd_Col[ii] == 1 and toNd_Col[ii] == 1:
            transType = 3
        transTpAll[ii] = transType
        # For boundary node, whether a within-cluster distractor exists
        tgt_in_cls   = node_which_cluster(from_nodes[ii], clsDef)
        clsDef_ii    = clsDef[np.squeeze(tgt_in_cls) == 1, :]
        dtrId_ii     = clusterResult.iloc[ii, objResp[1:]]
        whetherIn    = ismember(dtrId_ii, clsDef_ii)
        if np.sum(whetherIn) == 1:
            dtrInCls[ii] = 1
        else:
            dtrInCls[ii] = 0
        # Label the trial as correct or incorrect
        if clusterResult.iloc[ii, errorIdx] != 1 and clusterResult.iloc[ii, timeOIdx] != 1 and clusterResult.iloc[ii, fastIdx] != 1:
            iT_mark = 0 
        else:
            iT_mark = -1
        labCorAll[ii] = iT_mark 
        # Label the trials according to their correctness and respons time
        mouseTraj_arr = mouseTraj_trials[ii][0]
        mouseTraj_arr = mouseTraj_arr[:, [0, 1, 3]] # x-pos, y-pos and timePass
        mouse_bfCue   = np.sum(mouseTraj_arr[:, 2] <= grayoffT)
        respYes_arr   = respYes_trials[ii][0][:mouse_bfCue]
        respIncor     = np.sum(respYes_arr == -1)
        if respIncor >= 1:
            corrBfCue = -1 # tried to respond before cue appears, but failed
        else:
            corrBfCue = 0
        if clusterResult.iloc[ii, respRT] <= grayoffT:
            preResp_lab = 1
        else:
            preResp_lab = 0 
        # label trial types according to (1) preResp; (2) iT_mark; (3) corrBfCue
        if preResp_lab == 1 and iT_mark == 0 and corrBfCue == 0:
            markTrial = 0
        elif preResp_lab == 0 and iT_mark == 0 and corrBfCue == 0:
            markTrial = 1
        elif iT_mark == -1 and corrBfCue == -1:
            markTrial = 2
        elif iT_mark == -1 and corrBfCue == 0:
            markTrial = 3 
        labCorTimeAll[ii] = markTrial
        # recency
        rec_ii  = from_nodes[ii] == from_nodes[:ii]
        truePos = np.where(rec_ii)[0]
        if sum(rec_ii) == 0:
            recencyNodes[ii, 0] = nodeNum
            recencyNodes[ii, 1] = nodeNum
        elif sum(rec_ii) == 1:
            recencyNodes[ii, 0] = ii - truePos[-1]
            recencyNodes[ii, 1] = nodeNum
        elif sum(rec_ii) >= 2:
            recencyNodes[ii, 0] = ii - truePos[-1]
            recencyNodes[ii, 1] = ii - truePos[-2]
  
    subNo_Col = np.asarray([SubIdx] * clusterResult.shape[0]).reshape((clusterResult.shape[0], 1))     
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = subNo_Col)  
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = dtrInCls)     
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = dtrNoAll) 
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = transTpAll) 
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = labCorAll) 
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = labCorTimeAll) 
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = recencyNodes[:, 0]) 
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = recencyNodes[:, 1]) 
    
    if SubIdx == 0:
        clusterResult_pd = clusterResult
    else:
        clusterResult_pd = clusterResult_pd.append(clusterResult)

column_name = {0: 'trialCnt', 1: 'blockNo', 2: 'trialNo', 3: 'trialType', 4: 'targetObj', 5: 'respObj0', 6: 'respObj1', 7: 'respObj2', 8: 'respObj3', 9: 'moveTime', 10: 'RT', 
               11: 'dispRT', 12: 'errorIdx', 13: 'timeOutIdx', 14: 'bonus', 15: 'trialLt', 16: 'ObjX0', 17: 'ObjX1', 18: 'ObjX2', 19: 'ObjX3', 
               20: 'ObjY0', 21: 'ObjY1', 22: 'ObjY2', 23: 'ObjY3', 24: 'fastIdx', '25': 'fromNd', '26': 'toNd', '27': 'subNo', '28': 'dtrIn', '29': 'dtrNoCnt', '30': 'transType', 
               '31': 'trialMark', '32': 'trialMark2', '33': 'recencyL', '34': 'recencySL'} 
clusterResult_pd = clusterResult_pd.rename(columns = column_name)        
        
clusterResult_pd.subNo     = clusterResult_pd.subNo.astype('int64')
clusterResult_pd.trialCnt  = clusterResult_pd.trialCnt.astype('int64')
clusterResult_pd.blockNo   = clusterResult_pd.blockNo.astype('int64')
clusterResult_pd.trialNo   = clusterResult_pd.trialNo.astype('int64')
clusterResult_pd.trialType = clusterResult_pd.trialType.astype('int64')
clusterResult_pd.targetObj = clusterResult_pd.targetObj.astype('int64')
clusterResult_pd.respObj0  = clusterResult_pd.respObj0.astype('int64')
clusterResult_pd.respObj1  = clusterResult_pd.respObj1.astype('int64')
clusterResult_pd.respObj2  = clusterResult_pd.respObj2.astype('int64')
clusterResult_pd.respObj3  = clusterResult_pd.respObj3.astype('int64')
clusterResult_pd.errorIdx  = clusterResult_pd.errorIdx.astype('int64')
clusterResult_pd.timeOutIdx= clusterResult_pd.timeOutIdx.astype('int64')
clusterResult_pd.fastIdx   = clusterResult_pd.fastIdx.astype('int64')
clusterResult_pd.fromNd    = clusterResult_pd.fromNd.astype('int64')
clusterResult_pd.toNd      = clusterResult_pd.toNd.astype('int64')
clusterResult_pd.dtrIn     = clusterResult_pd.dtrIn.astype('int64')
clusterResult_pd.dtrNoCnt  = clusterResult_pd.dtrNoCnt.astype('int64')
clusterResult_pd.transType = clusterResult_pd.transType.astype('int64')
clusterResult_pd.trialMark = clusterResult_pd.trialMark.astype('int64')
clusterResult_pd.trialMark2 = clusterResult_pd.trialMark2.astype('int64')
clusterResult_pd.recencyL   = clusterResult_pd.recencyL.astype('int64')
clusterResult_pd.recencySL   = clusterResult_pd.recencySL.astype('int64')

path_or_buf = data_behv + 'clusterResult_pd.csv'
clusterResult_pd.to_csv(path_or_buf = path_or_buf)

del clusterResult_pd

## Reorganize mouseTraj

- At each time point, the distance and angle between the mouse cursor position and the center of the target image, as well as between the mouse cursor position and the center of the distractor image.
- If, at the current time point, the movement distances along both the x-axis and y-axis are each less than 1 cm compared to the previous time point, the mouse is considered not to have moved.

In [11]:
# Settimg the sampling points
timeIdx = np.arange(0, respDur+5*1/sampleRate, 1/sampleRate) # equivalent to: np.linspace(0, respDur, num=121); here 5 is an arbitrary number
timeIdx = pd.DataFrame(np.asarray(timeIdx).reshape(np.shape(timeIdx)[0], 1))

In [12]:
# Formatting the DataFrame and save as .csv for the subsequent usage
np.warnings.filterwarnings('ignore')
for SubIdx in range(0, subLen): 
    subjBv  = subj_listBv[SubIdx]
    subTit  = subjLab[SubIdx]
    subjDir = data_behv + subjBv + '/'
    # load the 'clusterResult.mat' which stores the x- and y-position of target image 
    behvMat = loadmat(subjDir + subjBv + 'clusterResult_Blocks.mat') # 'clusterResult'
    clusterResult = behvMat['clusterResult']
    clusterResult = pd.DataFrame(clusterResult)    
    trials_Col    = range(clusterResult.shape[0])
    respRT_Col    = clusterResult.iloc[trials_Col, respRT]
    rndHam_Col    = clusterResult.iloc[trials_Col, trialTp]
    # Labeling whether the from- and to-nodes are boundary nodes
    from_nodes    = clusterResult.iloc[trials_Col, objTgt]
    to_nodes      = clusterResult.iloc[trials_Col, objResp[0]]
    fromNd_Col    = np.zeros((clusterResult.shape[0], 1))
    toNd_Col      = np.zeros((clusterResult.shape[0], 1))
    fromNd_Col[np.logical_or.reduce((from_nodes==1, from_nodes==5, from_nodes==6, from_nodes==10, from_nodes==11, from_nodes==15))] = 1
    toNd_Col[np.logical_or.reduce((to_nodes==1, to_nodes==5, to_nodes==6, to_nodes==10, to_nodes==11, to_nodes==15))]               = 1
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = fromNd_Col)
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = toNd_Col)
    # Labeling whether there is a distractor that belongs to the same cluster as the target
    dtrInCls      = np.zeros((clusterResult.shape[0], 1))
    for ii in range(0, clusterResult.shape[0]):
        tgt_in_cls = node_which_cluster(from_nodes[ii], clsDef)
        clsDef_ii  = clsDef[np.squeeze(tgt_in_cls) == 1, :]
        dtrId_ii   = clusterResult.iloc[ii, objResp[1:]]
        whetherIn  = ismember(dtrId_ii, clsDef_ii)
        if np.sum(whetherIn) == 1:
            dtrInCls[ii] = 1
        else:
            dtrInCls[ii] = 0
    clusterResult.insert(loc = clusterResult.shape[1], column = str(clusterResult.shape[1]), value = dtrInCls)     
    # calculate the 'max distance' between the center point and the target's center
    maxDist_X     = clusterResult.iloc[trials_Col, imgPosX[0]] - centerX
    maxDist_X     = maxDist_X.values.reshape(np.shape(maxDist_X)[0], 1)
    maxDist_Y     = clusterResult.iloc[trials_Col, imgPosY[0]] - centerY
    maxDist_Y     = maxDist_Y.values.reshape(np.shape(maxDist_Y)[0], 1)
    maxXYdist     = np.squeeze([maxDist_X, maxDist_Y]).T
    dist_o_to_tgt = pd.Series(np.linalg.norm(maxXYdist, axis=1)) # default is L2 norm; this is a constant value, which equals to the radius
    for iBlock in range(0, nBlock):
        mouseTraj_cell   = loadmat(subjDir + subjBv + 'mouseTraj_trials_in_Blc' + str(iBlock + 1) + '.mat')
        mouseTraj_trials = mouseTraj_cell['mouseTraj_trials'] # np.shape(mouseTraj_trials)
        respYes_cell     = loadmat(subjDir + subjBv + 'respYes_trials_in_Blc' + str(iBlock + 1) + '.mat')
        respYes_trials   = respYes_cell['respYes_trials']
        trialIdx  = trialsInBlc[iBlock, :]
        error_blc = clusterResult.iloc[trialIdx[0] : trialIdx[1], errorIdx]
        timeO_blc = clusterResult.iloc[trialIdx[0] : trialIdx[1], timeOIdx]
        fast_blc  = clusterResult.iloc[trialIdx[0] : trialIdx[1], fastIdx]
        tgtX_blc  = clusterResult.iloc[trialIdx[0] : trialIdx[1], imgPosX[0]]
        tgtY_blc  = clusterResult.iloc[trialIdx[0] : trialIdx[1], imgPosY[0]]
        TgtId_blc = clusterResult.iloc[trialIdx[0] : trialIdx[1], objTgt]
        dtrNo_blc = clusterResult.iloc[trialIdx[0] : trialIdx[1], objResp]
        rt_blc    = clusterResult.iloc[trialIdx[0] : trialIdx[1], respRT]
        rsPosX_blc= clusterResult.iloc[trialIdx[0] : trialIdx[1], imgPosX[1:]] # x-position of distractors
        rsPosY_blc= clusterResult.iloc[trialIdx[0] : trialIdx[1], imgPosY[1:]] # y-position of distractors
        fromNd_blc= clusterResult.iloc[trialIdx[0] : trialIdx[1], fastIdx + 1]
        toNd_blc  = clusterResult.iloc[trialIdx[0] : trialIdx[1], fastIdx + 2]
        dtrIn_blc = clusterResult.iloc[trialIdx[0] : trialIdx[1], fastIdx + 3]
        rOh_blc   = clusterResult.iloc[trialIdx[0] : trialIdx[1], trialTp]     # random or hamiltonian path
        dist_o_to_tgt_blc = dist_o_to_tgt.iloc[trialIdx[0] : trialIdx[1]]
        # loop across the trials
        for iT in range(0, np.shape(mouseTraj_trials)[0]):
            mouseTraj_arr = mouseTraj_trials[iT][0]
            mouseTraj_arr = mouseTraj_arr[:, [0, 1, 3]] # x-pos, y-pos and timePass
            sampleLen     = len(mouseTraj_arr)
            subjNo_col    = np.asarray([SubIdx] * sampleLen).reshape((sampleLen, 1))
            blockNo_col   = np.asarray([iBlock] * sampleLen).reshape((sampleLen, 1))
            trialNo_col   = np.asarray([iT] * sampleLen).reshape((sampleLen, 1))
            sampleCnt_col = np.asarray(range(0, sampleLen)).reshape((sampleLen, 1))
            sampleTm_col  = timeIdx.iloc[0 : sampleLen]
            tgtX_iT       = tgtX_blc.iloc[iT]
            tgtY_iT       = tgtY_blc.iloc[iT]
            tgtX_col      = np.asarray([tgtX_iT] * sampleLen).reshape((sampleLen, 1))
            tgtY_col      = np.asarray([tgtY_iT] * sampleLen).reshape((sampleLen, 1))
            dtrNo_iT      = dtrNo_blc.iloc[iT, :]
            dtrNo_col     = np.asarray([np.shape(dtrNo_iT[dtrNo_iT != 0])[0]] * sampleLen).reshape((sampleLen, 1))
            if error_blc.iloc[iT] != 1 and timeO_blc.iloc[iT] != 1 and fast_blc.iloc[iT] != 1:
                iT_mark = 0 
            else:
                iT_mark = -1 # Incorrect trials
            iT_mark_col   = np.asarray([iT_mark] * sampleLen).reshape((sampleLen, 1))    
            maxDist_col   = np.asarray([dist_o_to_tgt.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            TgtId_col     = np.asarray([TgtId_blc.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            dtrLab_col    = np.asarray([dtrNo_iT] * sampleLen)
            rsPosX_col    = np.asarray([rsPosX_blc.iloc[iT, :]] * sampleLen)
            rsPosY_col    = np.asarray([rsPosY_blc.iloc[iT, :]] * sampleLen)
            # Labeling the trials where participants make incorrect response before cue appears
            mouse_bfCue   = np.sum(mouseTraj_arr[:, 2] <= grayoffT)
            respYes_arr   = respYes_trials[iT][0][:mouse_bfCue]
            respIncor     = np.sum(respYes_arr == -1)
            if respIncor >= 1:
                corrBfCue = -1 # tried to respond before cue appears, but failed
            else:
                corrBfCue = 0
            corrBfCueC_col= np.asarray([corrBfCue] * sampleLen).reshape((sampleLen, 1)) 
            # Labeling the transition directions: (1) within-within; (2) within-boundary; (3) boundary-within; (4) boundary-boundary
            if fromNd_blc.iloc[iT] == 0 and toNd_blc.iloc[iT] == 0:
                transType = 0
            elif fromNd_blc.iloc[iT] == 0 and toNd_blc.iloc[iT] == 1:
                transType = 1
            elif fromNd_blc.iloc[iT] == 1 and toNd_blc.iloc[iT] == 0:
                transType = 2
            elif fromNd_blc.iloc[iT] == 1 and toNd_blc.iloc[iT] == 1:
                transType = 3
            transType_col = np.asarray([transType] * sampleLen).reshape((sampleLen, 1))    
            fromNd_col    = np.asarray([fromNd_blc.iloc[iT]] * sampleLen).reshape((sampleLen, 1))   
            toNd_col      = np.asarray([toNd_blc.iloc[iT]] * sampleLen).reshape((sampleLen, 1))   
            # Add 'Angle' column to the previous dataframe:
            # firstly, calculate the angle between target stimuli and the zeroPos
            tgtXY    = np.array([tgtX_iT, tgtY_iT])
            zeroXY   = np.array([zeroPos[0], zeroPos[1]])
            centerXY = np.array([centerX, centerY])
            rad_t2o  = angle_to_onezero(tgtXY, zeroXY, centerXY)
            # secondly, calculate the angle between mouse trajectory sample and the zeroPos
            deltas_col  = np.zeros((sampleLen, 1))
            for iSam in range(0, sampleLen):
                mouXY   = np.array([mouseTraj_arr[iSam, 0], mouseTraj_arr[iSam, 1]])
                zeroXY  = np.array([zeroPos[0], zeroPos[1]])
                centerXY= np.array([centerX, centerY])
                rad_m2o = angle_to_onezero(mouXY, zeroXY, centerXY) # Sometimes the code output: 'RuntimeWarning: invalid value encountered in double_scalars', this is because Col2 is exactly at the (centerX, centerY)
                deltas_col[iSam] = rad_m2o - rad_t2o
            # ------ wrap angular differences into the interval [-pi, pi] ------
            deltas_col[deltas_col < -np.pi] = 2 * np.pi + deltas_col[deltas_col < -np.pi]
            deltas_col[deltas_col > np.pi]  = -2 * np.pi + deltas_col[deltas_col > np.pi]
            # label trials with RT smaller than 0.8s
            rt_col = np.asarray([rt_blc.iloc[iT]] * sampleLen).reshape((sampleLen, 1))
            if rt_blc.iloc[iT] <= grayoffT:
                preResp_col = np.ones((sampleLen, 1))
            else:
                preResp_col = np.zeros((sampleLen, 1))   
            # label trial types according to (1) preResp; (2) iT_mark; (3) corrBfCue
            if preResp_col[0] == 1 and iT_mark_col[0] == 0 and corrBfCueC_col[0] == 0:
                markTrial = 0
            elif preResp_col[0] == 0 and iT_mark_col[0] == 0 and corrBfCueC_col[0] == 0:
                markTrial = 1
            elif iT_mark_col[0] == -1 and corrBfCueC_col[0] == -1:
                markTrial = 2
            elif iT_mark_col[0] == -1 and corrBfCueC_col[0] == 0:
                markTrial = 3
            markTrial_col = np.asarray([markTrial] * sampleLen).reshape((sampleLen, 1))   
            # Mark whether there is another distractor that belongs to the same cluster as the target
            dtrIn_iT_col  = np.asarray([dtrIn_blc.iloc[iT]] * sampleLen).reshape((sampleLen, 1))   
            # random or hamiltonian path
            rOh_col       = np.asarray([rOh_blc.iloc[iT]] * sampleLen).reshape((sampleLen, 1))   
            # Calculate the distance between mouse and distractors
            disM2dtr = np.zeros((sampleLen, np.max(dtrNums))) # maximum number of distractors: 3
            for iD in range(np.shape(dtrNo_iT[dtrNo_iT != 0])[0] - 1):
                dtrX_iD  = rsPosX_blc.iloc[iT, iD]
                dtrY_iD  = rsPosY_blc.iloc[iT, iD]
                Xdist_iD  = mouseTraj_arr[:, 0] - dtrX_iD
                Ydist_iD  = mouseTraj_arr[:, 1] - dtrY_iD
                XYdist_iD = np.squeeze([Xdist_iD, Ydist_iD]).T
                dist_to_dtr = np.linalg.norm(XYdist_iD, axis=1)
                disM2dtr[:, iD] = dist_to_dtr / dist_o_to_tgt_blc.iloc[iT]

            # Concatenate the columns
            mouseTraj_conc = np.concatenate([subjNo_col, blockNo_col, trialNo_col, sampleCnt_col, sampleTm_col, mouseTraj_arr, tgtX_col, tgtY_col, dtrNo_col, iT_mark_col, markTrial_col, maxDist_col, deltas_col, rt_col, preResp_col,
                                             fromNd_col, toNd_col, transType_col, corrBfCueC_col, TgtId_col, dtrLab_col, rsPosX_col, rsPosY_col, dtrIn_iT_col, rOh_col, disM2dtr], axis=1)
            if iT == 0 and iBlock == 0:
                mouseTraj_pd  = pd.DataFrame(mouseTraj_conc, columns = ['subNo', 'blockNo', 'trialNo', 'sampleCnt', 'sampleTime', 'xPos', 'yPos', 'timePass', 'tgtX', 'tgtY', 'dtrNo', 'trialMark', 'trialMark2', 'maxDist', 'radians', 'RTs', 'preResp', 
                                                                        'fromNd', 'toNode', 'transType', 'respBfCue', 'targetObj', 'dtrObj0', 'dtrObj1', 'dtrObj2', 'dtrObj3', 'dtrX1', 'dtrX2', 'dtrX3', 'dtrY1', 'dtrY2', 'dtrY3', 'dtrInYes', 'randORham',
                                                                        'disM2dtr1', 'disM2dtr2', 'disM2dtr3']) 
            else:
                mouseTraj_pd2 = pd.DataFrame(mouseTraj_conc, columns = ['subNo', 'blockNo', 'trialNo', 'sampleCnt', 'sampleTime', 'xPos', 'yPos', 'timePass', 'tgtX', 'tgtY', 'dtrNo', 'trialMark', 'trialMark2', 'maxDist', 'radians', 'RTs', 'preResp', 
                                                                        'fromNd', 'toNode', 'transType', 'respBfCue', 'targetObj', 'dtrObj0', 'dtrObj1', 'dtrObj2', 'dtrObj3', 'dtrX1', 'dtrX2', 'dtrX3', 'dtrY1', 'dtrY2', 'dtrY3', 'dtrInYes', 'randORham',
                                                                        'disM2dtr1', 'disM2dtr2', 'disM2dtr3']) 
                mouseTraj_pd  = mouseTraj_pd.append(mouseTraj_pd2)
    
    mouseTraj_pd.subNo     = mouseTraj_pd.subNo.astype('int64')
    mouseTraj_pd.blockNo   = mouseTraj_pd.blockNo.astype('int64')
    mouseTraj_pd.trialNo   = mouseTraj_pd.trialNo.astype('int64')
    mouseTraj_pd.sampleCnt = mouseTraj_pd.sampleCnt.astype('int64')
    mouseTraj_pd.dtrNo     = mouseTraj_pd.dtrNo.astype('int64')
    mouseTraj_pd.trialMark = mouseTraj_pd.trialMark.astype('int64')
    mouseTraj_pd.trialMark2= mouseTraj_pd.trialMark2.astype('int64')
    mouseTraj_pd.preResp   = mouseTraj_pd.preResp.astype('int64')
    mouseTraj_pd.fromNd    = mouseTraj_pd.fromNd.astype('int64')
    mouseTraj_pd.toNode    = mouseTraj_pd.toNode.astype('int64')
    mouseTraj_pd.transType = mouseTraj_pd.transType.astype('int64')
    mouseTraj_pd.respBfCue = mouseTraj_pd.respBfCue.astype('int64')
    mouseTraj_pd.targetObj = mouseTraj_pd.targetObj.astype('int64')
    mouseTraj_pd.dtrObj0   = mouseTraj_pd.dtrObj0.astype('int64')
    mouseTraj_pd.dtrObj1   = mouseTraj_pd.dtrObj1.astype('int64')
    mouseTraj_pd.dtrObj2   = mouseTraj_pd.dtrObj2.astype('int64')
    mouseTraj_pd.dtrObj3   = mouseTraj_pd.dtrObj3.astype('int64')
    mouseTraj_pd.dtrInYes  = mouseTraj_pd.dtrInYes.astype('int64')
    mouseTraj_pd.randORham = mouseTraj_pd.randORham.astype('int64')
    
    mouseTraj_pd = mouseTraj_pd.reset_index()
    # mouseTraj_pd = mouseTraj_pd.drop(columns=['index'])
    # calculate the distance between mouse and target image, normalized with radius of the display region
    Xdist  = mouseTraj_pd.loc[:, 'xPos'] - mouseTraj_pd.loc[:, 'tgtX'] # series
    Xdist  = Xdist.values.reshape(np.shape(Xdist)[0], 1)
    Ydist  = mouseTraj_pd.loc[:, 'yPos'] - mouseTraj_pd.loc[:, 'tgtY']
    Ydist  = Ydist.values.reshape(np.shape(Ydist)[0], 1)
    XYdist = np.squeeze([Xdist, Ydist]).T
    dist_to_tgt = np.linalg.norm(XYdist, axis=1)
    mouseTraj_pd['disM2T'] = dist_to_tgt
    mouseTraj_pd['disM2T'] = mouseTraj_pd['disM2T'] / mouseTraj_pd['maxDist']
    
    # save the dataframe for each subjecta: df.to_save(r'Path where you want to store the exported CSV file/File Name.csv', index=False); if you wish to include the index, then simply remove "index=False" from the code
    path_or_buf  = subjDir + subjBv + 'mouseTraj_pd.csv'
    mouseTraj_pd.to_csv(path_or_buf=path_or_buf)
    
    del mouseTraj_pd, mouseTraj_pd2


In [13]:
# Read dataframe from all subjects and append into an unique file
for SubIdx in range(0, subLen):
    subjBv  = subj_listBv[SubIdx]
    subTit  = subjLab[SubIdx]
    subjDir = data_behv + subjBv + '/'
    path_or_buf  = subjDir + subjBv + 'mouseTraj_pd.csv'
    if SubIdx == 0:
        mouseTraj_pd_subj = pd.read_csv(filepath_or_buffer=path_or_buf, index_col=0) # ignore the untitled 1st colimn
    else:
        
        mouseTraj_pd_tmp  = pd.read_csv(filepath_or_buffer=path_or_buf, index_col=0) # ignore the untitled 1st colimn
        mouseTraj_pd_subj = mouseTraj_pd_subj.append(mouseTraj_pd_tmp)
mouseTraj_pd_subj = mouseTraj_pd_subj.drop(columns=['index'])
path_or_buf       = data_behv + 'mouseTraj_pd_subj.csv'
mouseTraj_pd_subj.to_csv(path_or_buf=path_or_buf)

In [9]:
path_or_buf       = data_behv + 'mouseTraj_pd_subj.csv'
mouseTraj_pd_subj = pd.read_csv(filepath_or_buffer = path_or_buf)
mouseTraj_pd_subj = mouseTraj_pd_subj.drop(columns = ['Unnamed: 0'])
mouseTraj_pd_subj

,subNo,blockNo,trialNo,sampleCnt,sampleTime,xPos,yPos,timePass,tgtX,tgtY,...,dtrX3,dtrY1,dtrY2,dtrY3,dtrInYes,randORham,disM2dtr1,disM2dtr2,disM2dtr3,disM2T
0,0,0,0,0,0.000000,1018.0,587.0,0.000080,947.985271,296.322244,...,0.000000,855.677756,0.000000,0.000000,0,1,0.969267,0.000000,0.000000,1.031630
1,0,0,0,1,0.016667,1017.0,593.0,0.016858,947.985271,296.322244,...,0.000000,855.677756,0.000000,0.000000,0,1,0.950520,0.000000,0.000000,1.050981
2,0,0,0,2,0.033333,1017.0,595.0,0.033480,947.985271,296.322244,...,0.000000,855.677756,0.000000,0.000000,0,1,0.943942,0.000000,0.000000,1.057703
3,0,0,0,3,0.050000,1017.0,600.0,0.050118,947.985271,296.322244,...,0.000000,855.677756,0.000000,0.000000,0,1,0.927518,0.000000,0.000000,1.074519
4,0,0,0,4,0.066667,1017.0,602.0,0.066844,947.985271,296.322244,...,0.000000,855.677756,0.000000,0.000000,0,1,0.920957,0.000000,0.000000,1.081249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2707673,23,6,199,69,1.150000,1280.0,683.0,1.149067,1287.136711,696.624869,...,1144.429004,839.564675,455.375131,312.435325,0,2,1.408416,1.958244,1.363144,0.053149
2707674,23,6,199,70,1.166667,1280.0,683.0,1.166202,1287.136711,696.624869,...,1144.429004,839.564675,455.375131,312.435325,0,2,1.408416,1.958244,1.363144,0.053149
2707675,23,6,199,71,1.183333,1280.0,683.0,1.181978,1287.136711,696.624869,...,1144.429004,839.564675,455.375131,312.435325,0,2,1.408416,1.958244,1.363144,0.053149
2707676,23,6,199,72,1.200000,1280.0,683.0,1.199576,1287.136711,696.624869,...,1144.429004,839.564675,455.375131,312.435325,0,2,1.408416,1.958244,1.363144,0.053149
